<DIV ALIGN=CENTER>

# Introduction to Machine Learning Pre-Processing
## Professor Robert J. Brunner
  
</DIV>  
-----
-----


## Introduction

In this IPython Notebook, we explore data pre-processing. This is a
large topic, so in this notebook we will focus on some basic concepts.
You should feel free to try completing additional tasks. To demonstrate
several common pre-processing steps, we will analyze the airline data
that was used in previous courses. These data are located at
`/home/data_scientist/data`, and we will specifically analyze the
`2001.csv` file. We will first read in a select group of columns with a
goal of constructing a Pandas DataFrame that has no missing data,
employs categorical values, and has a `DateTime` index.

We first set the Notebook to ignore warnings, since Pandas will specify
some warnings that are not relevant to this Notebook (for the curious,
we are adding columns, and Pandas is concerned we are modifying values).

-----

In [1]:
# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd 

# Change this to read a different file, for example
# /home/data_scientist/rppdm/data/2001.csv in a local Docker container.
#
# Note that the JupyterHub server has data from other years in the raw
# subdirectory.
#
filename = '/home/data_scientist/data/2001.csv'

# Read select columns for all rows.

ucs = (1, 2, 4, 14, 15, 16, 17, 18)
cnms = ['Month', 'Day', 'dTime', 'aDelay', 'dDelay', 'Depart', 'Arrive', 'Distance']

alldata = pd.read_csv(filename, header=0, na_values=['NA'], usecols=ucs, names=cnms)


-----

When we first read in a DataFrame, the first task should be to explore
the data. In this case, we will call the `describe` method, which
provides a statistical summary of the data. As shown below, we have six
numerical columns that have reasonable minimum and maximum values, as
well as basic statistical summaries.

-----

In [3]:
alldata.describe()

,Month,Day,dTime,aDelay,dDelay,Distance
count,5.967780e+06,5.967780e+06,5.736582e+06,5.723673e+06,5.736582e+06,5.967780e+06
mean,6.306294e+00,1.568320e+01,1.348705e+03,5.528249e+00,8.154837e+00,7.330293e+02
std,3.371688e+00,8.775346e+00,4.826860e+02,3.142929e+01,2.834847e+01,5.740716e+02
min,1.000000e+00,1.000000e+00,1.000000e+00,-1.116000e+03,-2.040000e+02,2.100000e+01
25%,3.000000e+00,8.000000e+00,NaN,NaN,NaN,3.130000e+02
50%,6.000000e+00,1.600000e+01,NaN,NaN,NaN,5.710000e+02
75%,9.000000e+00,2.300000e+01,NaN,NaN,NaN,9.800000e+02
max,1.200000e+01,3.100000e+01,2.400000e+03,1.688000e+03,1.692000e+03,4.962000e+03


-----

### Missing Values

One issue, however, is the different number of rows for the different
columns. In fact, there are three different values: 5967780, 5736582,
and 5723673. Since we will not want to perform any analysis on a row
that does not have a valid departure time (it isn't clear why we would
be missing departure times), we will first drop any row that does not
have a valid value in this column.

-----

In [4]:
# Drop any row that is missing a departure time
#
# axis = 0 means drop rows
# subset=['dTime'] means drop if the departure time is missing
# inplace=True means modify the existing DataFrame

alldata.dropna(axis=0, subset=['dTime'], inplace=True)

-----

Now we can explore the number of rows for each column by employing the
`count` method. In this case, we see that the `aDelay` column is still
missing some values. In this case, we can fill these missing values with
another value. There are a number of options for accomplishing this
task, including replacing with a default value (for example, we could
use the departure delay), interpolation, or even imputation (where we
might determine a mean value for flights in this distance range or from
the departure airport). In this case, however, we can simply assume the
arrival delay is zero for any missing value.

-----

In [5]:
# Now display number of 'good' values for each column.
alldata.count()

Month       5736582
Day         5736582
dTime       5736582
aDelay      5723673
dDelay      5736582
Depart      5736582
Arrive      5736582
Distance    5736582
dtype: int64

In [6]:
# Now replace missing values (which are all in Arrival Delay column)
# with 0, note we could use another value, such as the departure delay.

alldata.fillna(value=0, axis=0, inplace=True)

-----

We now create a new DataFrame to hold our final data set. In this case,
we start by copying over the three columns that won't change.

-----

In [7]:
# First create a new DataFrame
# For now, simply copy over the columns, but we could
# convert them to integers (for number, the number of minutes in the delay)
# and change data type to save memory.

newdata = alldata[['aDelay', 'dDelay', 'Distance']]

-----

### Categorical Values

In the original output from the `describe` method, neither the `Depart`
nor the `Arrive` columns were shown. Since they are interpreted as
string values, Pandas does not perform any statistical analysis of the
value. Since there are a limited number of airports that are contained
in our data, we can allow Pandas to map the Airport codes contained in
the `Depart` and `Arrive` columns to numerical representations by
employing [categorical][pdc] values. In this case, we can do this easily
by copying these columns over to our new DataFrame, and specifying that
they should be converted to type `categorical`.

To demonstrate the set of values that we will need to map to
categoricals, we can display all unique airport codes in the `Depart`
column.

-----
[pdc]: http://pandas.pydata.org/pandas-docs/stable/categorical.html

In [8]:
alldata['Depart'].unique()

array(['BWI', 'PHL', 'CLT', 'CLE', 'MKE', 'PIT', 'TPA', 'LGA', 'DFW',
       'BUF', 'BOS', 'ROC', 'DCA', 'MHT', 'CRW', 'MEM', 'DTW', 'CMH',
       'GSO', 'IAD', 'IAH', 'BHM', 'HPN', 'CHS', 'STL', 'AVP', 'ATL',
       'GSP', 'RDU', 'MCI', 'ORF', 'MYR', 'ALB', 'RIC', 'BNA', 'PVD',
       'BGM', 'TOL', 'SAV', 'ROA', 'IND', 'MDT', 'BTV', 'ELM', 'ITH',
       'JAX', 'PBI', 'SRQ', 'MSY', 'FLL', 'EWR', 'GRR', 'MSP', 'ORD',
       'ABE', 'LAS', 'BDL', 'MIA', 'MCO', 'CAE', 'ILM', 'SFO', 'SJU',
       'SDF', 'RSW', 'DAY', 'SYR', 'PNS', 'AVL', 'CAK', 'SBN', 'ERI',
       'HSV', 'CHA', 'SEA', 'TYS', 'FAY', 'LEX', 'STT', 'PWM', 'TRI',
       'STX', 'MDW', 'DAL', 'HOU', 'OKC', 'HRL', 'LIT', 'CRP', 'TUL',
       'LBB', 'MAF', 'ABQ', 'AMA', 'PHX', 'SAN', 'SMF', 'ISP', 'LAX',
       'AUS', 'PDX', 'ELP', 'OAK', 'ONT', 'SJC', 'JAN', 'RNO', 'BOI',
       'TUS', 'BUR', 'SAT', 'SLC', 'GEG', 'OMA', 'SNA', 'MSN', 'MBS',
       'FAR', 'ICT', 'GPT', 'VPS', 'DEN', 'TVC', 'FNT', 'GRB', 'GFK',
       'MOT', 'BIS',

In [9]:
# Now copy over the departure and Arrival columns, 
# but change data type to categoricals. 

newdata['Depart'] = alldata['Depart'].astype('category')
newdata['Arrive'] = alldata['Arrive'].astype('category')

-----

### DateTime

Lastly, we do not have a proper Date column nor any easily method for
handling times (such as Departure Time). Pandas has strong support for
[dates and times][pdt]. For our particular task, we can create a new
`DateTime` column. While it is generally easier to create a `DateTime`
column when reading a CSV file, it is not easy to account for missing
values during this process. In that case, one could read in the data,
clean missing values, and write the data back to a CSV (or other format)
file to enable the traditional data parsing to be performed.

In this case, however, we will instead create several new columns from
the original data set, and use these new columns to create a `DateTime`
column. First, we create a new `Year` column, which in this case we hard
code to `2001`, which is the year corresponding to these data. Second,
we copy over the `Month` and `Day` columns. Finally, we convert the
encoded departure time column to an `Hour` and `Min`. Before any new
processing, we first display the last few rows of our new DataFrame
before computing summary statistics for the new DataFrame to verify our
work.

-----

[pdt]: http://pandas.pydata.org/pandas-docs/stable/timeseries.html

In [10]:
newdata['Year'] = 2001
newdata['Month'] = alldata.Month
newdata['Day'] = alldata.Day
newdata['Hour'] = (alldata.dTime/100.).astype(int)
newdata['Min'] = (alldata.dTime - 100*(alldata.dTime/100.).astype(int)).astype(int)

In [11]:
newdata.tail()

,aDelay,dDelay,Distance,Depart,Arrive,Year,Month,Day,Hour,Min
5967775,4.0,4.0,1189,ONT,DFW,2001,12,14,7,4
5967776,3.0,8.0,1189,ONT,DFW,2001,12,15,7,8
5967777,-8.0,-4.0,1189,ONT,DFW,2001,12,16,6,56
5967778,-4.0,-4.0,1189,ONT,DFW,2001,12,17,6,56
5967779,3.0,9.0,1189,ONT,DFW,2001,12,18,7,9


In [12]:
newdata.describe()

,aDelay,dDelay,Distance,Year,Month,Day,Hour,Min
count,5.736582e+06,5.736582e+06,5.736582e+06,5736582.0,5.736582e+06,5.736582e+06,5.736582e+06,5.736582e+06
mean,5.515809e+00,8.154837e+00,7.356117e+02,2001.0,6.291336e+00,1.571203e+01,1.319109e+01,2.959601e+01
std,3.139500e+01,2.834847e+01,5.749639e+02,0.0,3.381300e+00,8.827161e+00,4.830028e+00,1.781198e+01
min,-1.116000e+03,-2.040000e+02,2.100000e+01,2001.0,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,-9.000000e+00,-3.000000e+00,3.140000e+02,2001.0,3.000000e+00,8.000000e+00,9.000000e+00,1.400000e+01
50%,-2.000000e+00,0.000000e+00,5.760000e+02,2001.0,6.000000e+00,1.600000e+01,1.300000e+01,3.000000e+01
75%,1.000000e+01,6.000000e+00,9.840000e+02,2001.0,9.000000e+00,2.300000e+01,1.700000e+01,4.500000e+01
max,1.688000e+03,1.692000e+03,4.962000e+03,2001.0,1.200000e+01,3.100000e+01,2.400000e+01,5.900000e+01


----

Now we can turn our attention to creating a `DateTime` column in our new
DataFrame. However, in the output of the `describe` method, you should
notice that the `Hour` column has a maximum of 24 hours. Our method of
creating a `DateTime` value employs the Python [`strftime`][pst] string
date/time representation. This representation does not allow values
higher than 23 for an hour, hence we have to modify our data. For this
demonstration, we will simply subtract one minute. In a production
system, you might instead convert the `DateTime` to the next day.

Once these columns have been modified appropriately, we create a
DateTime by using the `to_datetime` method. We first combine our date
and time DataFrame columns into a new string value, and specify the
appropriate format string to decode these values. After creating this
new column, we indicate that it should be used as the index for the new
DateFrame. Finally, we display the last few rows of the new DataFrame to
show the values.

-----
[pst]: http://strftime.org

In [13]:
newdata.loc[newdata.Hour == 24, 'Min'] = 59
newdata.loc[newdata.Hour == 24, 'Hour'] = 23

In [14]:
newdata['DTI'] = pd.to_datetime(newdata.Year * 100000000 + 
                                newdata.Month * 1000000 + 
                                newdata.Day * 10000 + 
                                newdata.Hour * 100 + 
                                newdata.Min, format="%Y%m%d%H%M")

In [15]:
newdata.set_index('DTI', inplace=True)

In [16]:
newdata.tail()

,aDelay,dDelay,Distance,Depart,Arrive,Year,Month,Day,Hour,Min
DTI,,,,,,,,,,
2001-12-14 07:04:00,4.0,4.0,1189,ONT,DFW,2001,12,14,7,4
2001-12-15 07:08:00,3.0,8.0,1189,ONT,DFW,2001,12,15,7,8
2001-12-16 06:56:00,-8.0,-4.0,1189,ONT,DFW,2001,12,16,6,56
2001-12-17 06:56:00,-4.0,-4.0,1189,ONT,DFW,2001,12,17,6,56
2001-12-18 07:09:00,3.0,9.0,1189,ONT,DFW,2001,12,18,7,9


-----

At this point we have a DataFrame that contains a DateTime Index, no
missing values, and has the airport codes represented by categoricals.
We could save this data for latter analysis, or being applying learning
algorithms.

-----